In [1]:
import param
import panel as pn
import pandas as pd
import time
import requests

from io import StringIO


import plotly.express as px

pn.extension('plotly', 'tabulator', sizing_mode="stretch_width")

In [14]:
#API secret 
from lib.api_secret import API_KEY

In [15]:
class AudioDashboard(param.Parameterized):
    data = param.Dict()
  
    file_input = param.Parameter()
    
    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.TextInput(), **params)
        self.transcript = pn.pane.Str(max_width = 600, height_policy = "fit")
        self.download = pn.widgets.FileDownload(name="Download transcript", filename="transcript.txt", callback=self._download_callback, button_type="primary")

    @pn.depends("file_input.value", watch=True)
    def _parse_file_input(self):
        print("parse input ")
        print(self.file_input)
        value = self.file_input.value
        if value:
            print("in parse")
            self.data = self.get_and_analyze_transcript()
            print(self.data)
            
        else:
            print("error")

    @pn.depends('data', watch=True)
    def get_transcript(self):
        print("get transcript ")
        self.transcript.object = self.data["text"]
        
        
    def _download_callback(self):
        print("download")
        if self.data is not None:
            buffer = StringIO()
            buffer.write(self.data["text"])
            buffer.seek(0)
            print("return buffer")

            return buffer  
        else:
            return
        
        
    def poll(self):
        polling_endpoint = transcript_endpoint + '/' + self.transcript_id
        polling_response = requests.get(polling_endpoint, headers=self.headers)
        self.data = polling_response.json()
        
    def get_and_analyze_transcript(self):
        audio_url = self.file_input.value
                
        if audio_url:
            print("yessss")
            api_key_value =API_KEY

            endpoint = "https://api.assemblyai.com/v2/transcript"
            audio_url_json = {
                "audio_url": audio_url,
                "sentiment_analysis": True,
                "auto_highlights": True,
                "iab_categories": True,
                "auto_chapters": True
            }
            headers = {
                "authorization": api_key_value,
                "content-type": "application/json"
            }
            transcript_response = requests.post(endpoint, json=audio_url_json, headers=headers)

            result_endpoint = endpoint + '/' + transcript_response.json()["id"]
            headers_auth = {
                "authorization": api_key_value,
            }
            response = requests.get(result_endpoint, headers=headers_auth)
            while response.json()['status'] != "completed":
                response = requests.get(result_endpoint, headers=headers_auth)
                time.sleep(3)
                
            return response.json()

        else:
            print("no")
            return
        
    def view(self):
        return pn.Column(
            "## Transcript",
            self.transcript,
        )
    
audio_app = AudioDashboard()

audio_app_view = audio_app.view()
audio_app_view

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Str(None, height_policy='fit', max_width=600, sizing_mode='stretch_width')

In [17]:
description = """
This is my app :)
"""

component = pn.Column(
    description,
    audio_app_view,
    sizing_mode='stretch_both'
)
component

template = pn.template.FastListTemplate(
    title='Audio Content Explorer', 
    sidebar=[pn.pane.Markdown("### Input a link:"), 
             audio_app.file_input, 
             audio_app.download],
    main=[component],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

template.servable()

parse input 
TextInput(sizing_mode='stretch_width', value='https://github.com/AyeshG..., value_input='https://github.com/AyeshG...) 
in parse 
yessss 
get transcript 
{'id': 'of5rrf84hi-31c4-4833-9152-8ef260356261', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'completed', 'audio_url': 'https://github.com/AyeshGK/audio_analyzer/blob/main/How_I_Would_Learn_to_Code.mp3?raw=true', 'text': "I recently got an amazing opportunity. I've been learning jujitsu, and a friend offered to coach me in exchange for teaching him how to code. At this point in my life where I'm trying to improve my jiu jitsu as quickly as possible, this was an incredible trade for me. In my personal opinion, I was all for until I had to think about how I'd actually go about teaching someone how to code starting from nothing, I quickly realized that this was going to be a lot harder than I thought. After all, it's been almost ten years since I wrote my very first line of code. That's a pretty long time. Now, this video is about the very specific and detailed plan that I wrote for my friend to teach me the basics of coding for data science. Since I did all the work for him, I figured I might as well share the details with you as well. Here. There have been plenty of other great videos similar to this about how to start coding, and I've linked many of them in the description. Again, those are great, but I think this is a bit different because it's more focused on coding for data and it uses a very specific real world case study. At the end, I also highlight the three most important things a new programmer should know. So you definitely want to stick around for that. Let's start with the language that we should start learning. I know if you're working with data, you generally have two good choices. They would either start with Python or R. Let's quickly compare the two languages data Science Style when looking at the Stack Overflow developer survey, we find that about 48% of respondents use Python, while only about 5% user. In industry, Python is a far more common language. Let's now look more specifically a data scientist, not just all Stack Overflow users who come from a variety of backgrounds. On Kaggle, my favorite platform for data science learning and projects, we can see that around 84.2% of respondents use Python, while only 25% use R. Since these add up to more than 100%, we can assume that some data scientists are using both languages. Finally, let's look at these two languages over time. We can see here from 2020 to 2021, Python has gained an adoption. While our usage has seen some attrition, to me, this is a no brainer. I recommended my friend to start learning Python first. That language is more popular, and it's continuing to grow. This isn't to say R is bad. It's just to say that I think Python is growing and will be a more valuable skill to have for the job market in general. I also personally find it a bit more intuitive and flexible than our. Now that we've chosen our language as Python, what coding skills should my friend learn first? Should he focus on variables? Logic functions? Algorithms? Classes? My mind was absolutely spinning when I first addressed this. It took me a second, but I had this weird chaotic flashback to my first days of learning programming myself. I didn't even know where to start with getting Python on my own computer. I just now got excited about teaching the functional part of programming, and I completely forgot about the basics of setting up your environment for success. The next thing I did was give my friend some of my favorite resources on choosing an environment and getting the right version of Python on his computer. Honestly, I was a bit torn on this too. So, for example, what IDE should have used? Does he even know what IDE stands for? For those who don't know and who are new, ID stands for Integrated Development Environment. So this is essenti

FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [2208630220880] Column(sizing_mode='stretch_both')
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Str(None, height_policy='fit', max_width=600, sizing_mode='stretch_width')
    [2208630887136] Markdown(str, sizing_mode='stretch_width')
    [2208629304480] TextInput(sizing_mode='stretch_width', value_input='https://github.com/AyeshG...)
    [2208629305152] FileDownload(button_type='primary', callback=<bound method AudioDashboa..., filename='transcript.txt', label='Download transcript.txt', name='Download transcript', sizing_mode='stretch_width')